In [1]:
import os
normal_images = os.listdir('/kaggle/input/data-for-classification/train/non-malignant')
pneumonia_images = os.listdir('/kaggle/input/data-for-classification/train/malignant')

print('Тренировочная выборка:')
print('Без  узлов ',  len(normal_images))
print('Есть злокачественные узлы ',  len(pneumonia_images))
print()

normal_images = os.listdir('/kaggle/input/data-for-classification/val/non-malignant')
pneumonia_images = os.listdir('/kaggle/input/data-for-classification/val/malignant')

print('Без  узлов ',  len(normal_images))
print('Есть злокачественные узлы ',  len(pneumonia_images))
print()

normal_images = os.listdir('/kaggle/input/data-for-classification/test/non-malignant')
pneumonia_images = os.listdir('/kaggle/input/data-for-classification/test/malignant')

print('Без  узлов ',  len(normal_images))
print('Есть злокачественные узлы ',  len(pneumonia_images))
print()

Тренировочная выборка:
Без  узлов  6619
Есть злокачественные узлы  4560



Без  узлов  2113
Есть злокачественные узлы  257



Без  узлов  2030
Есть злокачественные узлы  236



In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2025-05-15 12:56:49.420044: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747313809.608219      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747313809.661944      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models
bbox_width=80



def build_model(input_shape=(bbox_width, bbox_width, 3)):
    '''
    base_model = EfficientNetB0(
        include_top=False,
        input_shape=input_shape,
        weights=None  
    )
    '''
    base_model = InceptionV3(
    include_top=False,
    weights='imagenet',
    input_shape=(80, 80, 3)
)
    for layer in base_model.layers[:-100]:
        layer.trainable = False
    inputs = tf.keras.Input(shape=input_shape)
    x = base_model(inputs, training=True)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.Model(inputs, x)
    return model


In [4]:
model = build_model(input_shape=(bbox_width, bbox_width, 3))
model.summary()

I0000 00:00:1747313825.055565      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1747313825.056323      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


       0/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step

 9388032/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

24952832/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

40108032/87910968 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

...

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 80, 80, 3)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ inception_v3 (Functional)            │ (None, 1, 1, 2048)          │      21,802,784 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 2048)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │           2,049 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,804,833 (83.18 MB)

 Trainable params: 13,626,113 (51.98 MB)

 Non-trainable params: 8,178,720 (31.20 MB)

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.metrics import Precision, Recall
bbox_width=80







optimizer=tf.keras.optimizers.Adam(learning_rate=0.005)

loss='binary_crossentropy'

model.compile(loss=loss,
              optimizer=optimizer,
                  metrics=[  
        Precision(name='precision'),
        Recall(name='recall')])


train_data_dir = '/kaggle/input/data-for-classification/train'
val_data_dir = '/kaggle/input/data-for-classification/val'
test_data_dir = '/kaggle/input/data-for-classification/test'


train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        #target_size=(1024, 1024),
        target_size=(bbox_width, bbox_width),
        batch_size=32,
        color_mode='rgb',
        class_mode='binary',
        classes=["non-malignant", "malignant"] )

validation_generator = val_datagen.flow_from_directory(
        val_data_dir,
        target_size=(bbox_width, bbox_width),
        batch_size=32,
        color_mode='rgb',
        class_mode='binary',
        classes=["non-malignant", "malignant"])

test_generator = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(bbox_width, bbox_width),
        batch_size=32,
        color_mode='rgb',
        class_mode='binary',
        classes=["non-malignant", "malignant"])

checkpoint_filepath = 'saved_model/best_model.keras'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_loss',
    mode='min',
    save_best_only=True)



history=model.fit(train_generator, epochs=50, validation_data=validation_generator, callbacks=[model_checkpoint_callback], verbose=2)

Found 11179 images belonging to 2 classes.


Found 2370 images belonging to 2 classes.


Found 2266 images belonging to 2 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


I0000 00:00:1747313876.912508      62 service.cc:148] XLA service 0x79d3d0072a50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1747313876.913263      62 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1747313876.913288      62 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5


In [ ]:
model = tf.keras.models.load_model('/kaggle/working/saved_model/best_model.keras')

In [ ]:
print(train_generator.class_indices)